# Imports

In [2]:
import scipy
from scipy.optimize import linprog
#scipy.__version__

import requests
import pandas as pd

# JSON Live Station Data

In [3]:
station_info_response = requests.get("https://gbfs.citibikenyc.com/gbfs/en/station_information.json").json()
station_info = pd.DataFrame(station_info_response['data']['stations'])[["station_id","name","lon","lat"]]

station_status_response = requests.get("https://gbfs.citibikenyc.com/gbfs/en/station_status.json").json()
station_status = pd.DataFrame(station_status_response['data']['stations'])[["station_id","num_docks_available","num_bikes_available"]]

In [4]:
station_info.head(5)

,station_id,name,lon,lat
0,72,W 52 St & 11 Ave,-73.993929,40.767272
1,79,Franklin St & W Broadway,-74.006667,40.719116
2,82,St James Pl & Pearl St,-74.000165,40.711174
3,83,Atlantic Ave & Fort Greene Pl,-73.976323,40.683826
4,116,W 17 St & 8 Ave,-74.001497,40.741776


In [5]:
station_status.head(5)

,station_id,num_docks_available,num_bikes_available
0,72,17,0
1,79,29,2
2,82,0,26
3,83,33,24
4,116,11,58


In [6]:
station_merged = pd.merge(station_status,station_info,how='left')

In [7]:
station_merged.head(5)

,station_id,num_docks_available,num_bikes_available,name,lon,lat
0,72,17,0,W 52 St & 11 Ave,-73.993929,40.767272
1,79,29,2,Franklin St & W Broadway,-74.006667,40.719116
2,82,0,26,St James Pl & Pearl St,-74.000165,40.711174
3,83,33,24,Atlantic Ave & Fort Greene Pl,-73.976323,40.683826
4,116,11,58,W 17 St & 8 Ave,-74.001497,40.741776


In [8]:
print(station_info.shape)
print(station_status.shape)
print(station_merged.shape)

(1791, 4)
(1791, 3)
(1791, 6)


# LP model with a test data_set
The LP with 1000 stations takes considerably long. The Heuristic model should be used for less optimized but a faster model.

In [124]:
df = pd.read_csv("/Users/dehaay/Desktop/BikeShare Project/experiment/allocation_mock_data.csv").iloc[3:11]

In [125]:
df

,Station names,capacity %,bike count in an hour,station capacity,x cord,y cord
3,D,78%,69,89,66,32
4,E,63%,40,64,95,69
5,F,102%,49,48,4,75
6,G,93%,65,70,52,90
7,H,75%,56,75,18,15
8,I,6%,3,47,89,98
9,J,71%,63,89,74,42
10,K,84%,57,68,58,37


In [126]:
equilibrium_percent = df["bike count in an hour"].sum() / df["station capacity"].sum()
equilibrium_percent

0.730909090909091

In [127]:
df["give_away_stock"] = -(round(df["station capacity"] * equilibrium_percent) - df["bike count in an hour"])

In [128]:
df

,Station names,capacity %,bike count in an hour,station capacity,x cord,y cord,give_away_stock
3,D,78%,69,89,66,32,4.0
4,E,63%,40,64,95,69,-7.0
5,F,102%,49,48,4,75,14.0
6,G,93%,65,70,52,90,14.0
7,H,75%,56,75,18,15,1.0
8,I,6%,3,47,89,98,-31.0
9,J,71%,63,89,74,42,-2.0
10,K,84%,57,68,58,37,7.0


In [129]:
import itertools
cross_product = list(itertools.product(df["Station names"],df["Station names"]))
station_combinations = ["_".join([i,b]) for i,b in cross_product if i != b]

In [130]:
cross_product = list(itertools.product(zip(df["Station names"],df["x cord"],df["y cord"]),zip(df["Station names"],df["x cord"],df["y cord"]) ))
x_cord_diff = [ abs(i[1]-b[1]) for i,b in cross_product if i[0] != b[0]]
y_cord_diff = [ abs(i[2]-b[2]) for i,b in cross_product if i[0] != b[0]]

In [131]:
distance_combinations = [sum(i) for i in zip(x_cord_diff, y_cord_diff )] 

In [132]:
len(station_combinations)

56

In [133]:
len(distance_combinations)

56

## LP formulation

Objective function

\begin{equation*}
{\begin{array}{1ll}
\small\text{Minimize} \Large\sum\limits_{i}^{A} \sum\limits_{j}^{A}S_{ij} \times x_{ij} \\
& \small\text{$S_{ij}$ is transfer realtion between station i and j where i $\neq$ j} \\[2pt]
& \small\text{A: all stations} \\[2pt]
& \small\text{x: distance between station i and j} \\[2pt]
\end{array}}
\end{equation*}



\begin{cases}
      \Large\sum\limits_{j}^{A} S_{ji} - S_{ij}  = D_i & D_i \leq 0 \\
       \Large\sum\limits_{j}^{A} S_{ij} - S_{ji}  = D_i  & D_i \geq 0 
\end{cases}

\begin{equation*}
{\begin{array}{1ll}
\small\text{$S_{ij}$ is transfer realtion between station i and j where i $\neq$ j} \\
\small\text{D_i: Bike demand of station i for reaching the equilibrium}
\end{array}}
\end{equation*}


In [134]:
## ============================================== ##
## Decision Variables                             ## 
## ============================================== ##
VarName = station_combinations

## ============================================== ##
## Objective function coefficients (minimization) ## 
## ============================================== ##
c = distance_combinations

print(len(c))

56


In [135]:
## ============================================== ##
## Equality constraints                           ## 
## ============================================== ##
import numpy as np
A_eq, b_eq = [], []

In [136]:
for i in range(len(df.index)):
    df_instance = df.iloc[i]
    give_away_stock = df_instance["give_away_stock"]
    station_name = df_instance["Station names"]
    
    temp_array = np.array([0] * len(c))
    reverse_indexes = [count for count,i in enumerate(VarName) if i.split("_")[1] == station_name ]
    regular_indexes = [count for count,i in enumerate(VarName) if i.split("_")[0] == station_name ]
    

    if give_away_stock < 0:

        temp_array[reverse_indexes] = 1
        temp_array[regular_indexes] = -1
        
        A_eq.append(temp_array.tolist())
        b_eq.append(abs(give_away_stock))
        
    elif give_away_stock > 0:
        
        temp_array[reverse_indexes] = -1
        temp_array[regular_indexes] = 1
        
        A_eq.append(temp_array.tolist())
        b_eq.append(abs(give_away_stock))
 
    else:
        temp_array[reverse_indexes] = 1
        temp_array[regular_indexes] = -1
        
        A_eq.append(temp_array.tolist())
        b_eq.append(0)


In [137]:
## ============================================== ##
## Nonnegativity                                  ## 
## ============================================== ##
lbs = [0]*len(c)
ubs = [None]*len(c)
bounds = [(lb, ub) for lb, ub in zip(lbs, ubs)]

In [138]:
## ============================================== ##
## Print constraints                              ## 
## ============================================== ##
for row, b in zip(A_eq, b_eq):
    print( " + ".join([repr(ele) + "*" + vn for ele, vn in zip(row, VarName) if abs(ele) > 0.0]) + " = " + repr(b) )


1*D_E + 1*D_F + 1*D_G + 1*D_H + 1*D_I + 1*D_J + 1*D_K + -1*E_D + -1*F_D + -1*G_D + -1*H_D + -1*I_D + -1*J_D + -1*K_D = 4.0
1*D_E + -1*E_D + -1*E_F + -1*E_G + -1*E_H + -1*E_I + -1*E_J + -1*E_K + 1*F_E + 1*G_E + 1*H_E + 1*I_E + 1*J_E + 1*K_E = 7.0
-1*D_F + -1*E_F + 1*F_D + 1*F_E + 1*F_G + 1*F_H + 1*F_I + 1*F_J + 1*F_K + -1*G_F + -1*H_F + -1*I_F + -1*J_F + -1*K_F = 14.0
-1*D_G + -1*E_G + -1*F_G + 1*G_D + 1*G_E + 1*G_F + 1*G_H + 1*G_I + 1*G_J + 1*G_K + -1*H_G + -1*I_G + -1*J_G + -1*K_G = 14.0
-1*D_H + -1*E_H + -1*F_H + -1*G_H + 1*H_D + 1*H_E + 1*H_F + 1*H_G + 1*H_I + 1*H_J + 1*H_K + -1*I_H + -1*J_H + -1*K_H = 1.0
1*D_I + 1*E_I + 1*F_I + 1*G_I + 1*H_I + -1*I_D + -1*I_E + -1*I_F + -1*I_G + -1*I_H + -1*I_J + -1*I_K + 1*J_I + 1*K_I = 31.0
1*D_J + 1*E_J + 1*F_J + 1*G_J + 1*H_J + 1*I_J + -1*J_D + -1*J_E + -1*J_F + -1*J_G + -1*J_H + -1*J_I + -1*J_K + 1*K_J = 2.0
-1*D_K + -1*E_K + -1*F_K + -1*G_K + -1*H_K + -1*I_K + -1*J_K + 1*K_D + 1*K_E + 1*K_F + 1*K_G + 1*K_H + 1*K_I + 1*K_J = 7.0


In [139]:
## ============================================== ##
## Print optimization result                      ## 
## ============================================== ##
res = linprog(c, A_eq = A_eq, b_eq = b_eq, bounds = bounds, method = 'highs')
print(res.message)
print("Minimized objective function value := ", res.fun)
print("Optimal solution:")
for x, vn in zip(res.x, VarName):
    print(vn, " := ", x)

Optimization terminated successfully.
Minimized objective function value :=  2993.0
Optimal solution:
D_E  :=  2.0
D_F  :=  0.0
D_G  :=  0.0
D_H  :=  0.0
D_I  :=  0.0
D_J  :=  2.0
D_K  :=  0.0
E_D  :=  0.0
E_F  :=  0.0
E_G  :=  0.0
E_H  :=  0.0
E_I  :=  0.0
E_J  :=  0.0
E_K  :=  0.0
F_D  :=  0.0
F_E  :=  0.0
F_G  :=  0.0
F_H  :=  0.0
F_I  :=  14.0
F_J  :=  0.0
F_K  :=  0.0
G_D  :=  0.0
G_E  :=  0.0
G_F  :=  0.0
G_H  :=  0.0
G_I  :=  14.0
G_J  :=  0.0
G_K  :=  0.0
H_D  :=  0.0
H_E  :=  0.0
H_F  :=  0.0
H_G  :=  0.0
H_I  :=  1.0
H_J  :=  0.0
H_K  :=  0.0
I_D  :=  0.0
I_E  :=  0.0
I_F  :=  0.0
I_G  :=  0.0
I_H  :=  0.0
I_J  :=  0.0
I_K  :=  0.0
J_D  :=  0.0
J_E  :=  0.0
J_F  :=  0.0
J_G  :=  0.0
J_H  :=  0.0
J_I  :=  0.0
J_K  :=  0.0
K_D  :=  0.0
K_E  :=  5.0
K_F  :=  0.0
K_G  :=  0.0
K_H  :=  0.0
K_I  :=  2.0
K_J  :=  0.0


# Allocation Report

$S_1\_S_2 = N$ is read transfer N bikes from $S_1$ to $S_2 $

In [140]:
[[var,value] for value,var in zip(res.x, VarName) if value > 0]

[['D_E', 2.0],
 ['D_J', 2.0],
 ['F_I', 14.0],
 ['G_I', 14.0],
 ['H_I', 1.0],
 ['K_E', 5.0],
 ['K_I', 2.0]]